In [15]:
import pandas as pd
import numpy as np
from viz import show_field
from tools import make_move

In [2]:
df = pd.read_csv('resources/test.csv', index_col='id')

In [3]:
from GeneticSolver import GeneticSolver
from MPGeneticSolver import MPGeneticSolver

In [4]:
mpgs = MPGeneticSolver(early_stopping=False)

In [20]:
solution_df = pd.DataFrame([], columns=['id', 'score'] + ['start.'+ str(_) for _ in range(1, 401)], dtype=int)

In [26]:
for col in solution_df.columns:
    solution_df[col] = solution_df[col].astype(np.int32)

In [28]:
running_avg, n = 0, 0
log = []
best, worst = None, None
for id, (idx, row) in zip(df.index, df.iterrows()):
    delta, Y = row.values[0], row.values[1:].reshape((20, 20)).astype('uint8')
    solution = mpgs.solve(Y, delta, return_all=False)

    board, score = solution
    flat_board = np.insert(board.ravel(), 0, id)
    flat_board = np.insert(flat_board, 1, int(score * 100))
    solution_df = solution_df.append(pd.Series(flat_board, index=solution_df.columns), ignore_index=True)

    log.append((idx, score))
    if best is None or best[1] < score:
        best = (idx, score)
    if worst is None or worst[1] > score:
        worst = (idx, score)
    n += 1
    running_avg = (running_avg * (n - 1) + score) / n
    print(f"{idx} is solved with score {score}. Average score: {running_avg}")
    if idx > 0:
        break
print("Best score:", best)
print("Worst score:", worst)

1 is solved with score 1.0. Average score: 1.0
Best score: (1, 1.0)
Worst score: (1, 1.0)


In [29]:
solution_df

,id,score,start.1,start.2,start.3,start.4,start.5,start.6,start.7,start.8,...,start.391,start.392,start.393,start.394,start.395,start.396,start.397,start.398,start.399,start.400
0,1,100,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [30]:
solution_df.to_csv('test_save.csv', index=False)